In [1]:
import numpy as np
import os
import csv
import runtests

In [2]:
def return_one(startdisk, zfsname, csvfilename, ip, skip_num=0):
    with open(csvfilename) as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        next(reader)
        for _ in range(skip_num):
            next(reader)
        row = next(reader)
        testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs = row
        print (testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)
        print ("++++++++++++++++++++++++++++")
        print (testname)
        print ("++++++++++++++++++++++++++++")
        numberdisks = int(numberdisks)
        raidmode = int(raidmode)
        raid0 = int(raid0)
        return runtests.make_commands(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime, testname, ip, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)

In [12]:
cmds = return_one("c", "tank", "config.csv", "192.168.169.207", skip_num=6)

z2_one_8_single 8 6 0 12  32m off 500g True 200 16m 32m 64m 1 2 4 1 4 8
++++++++++++++++++++++++++++
z2_one_8_single
++++++++++++++++++++++++++++


In [13]:
for cmd in cmds:
    print(cmd)

mkdir z2_one_8_single
echo -------------------------------
echo make zfs
echo -------------------------------
zpool create tank -o ashift=12 raidz2 sdc sdd sde sdf sdg sdh sdi sdj
echo -------------------------------
echo test zfs
echo -------------------------------
./bench_fio  --type directory --quiet -m write read --loops 1 --target /tank -o z2_one_8_single_zfs -b 16m 32m 64m --iodepth 1 2 4 --numjobs 1 4 8 --size 500g --runtime 200
echo -------------------------------
echo make lustre
echo -------------------------------
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --reformat --mgs --mdt mgsmdt/mgsmdt
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --ost --mgsnode=192.168.169.207@tcp tank/ost0
mount -t lustre mgsmdt/mgsmdt /mnt/mgsmdt
mount -t lustre tank/ost0 /mnt/ost
mount -t lustre 192.168.169.207@tcp:/test /mnt/lustre
echo -------------------------------
echo test lustre
echo -------------------------------
./bench_fio  --type directory --quiet -m write read --loo

In [5]:
# csv stuff is in makezpool.py

In [6]:
# STATICS
RAID_CONFIGS = {
    5:"raidz",
    6:"raidz2",
    7:"raidz3",
    0:"",
    1:"mirror"
}

ALPHABET = alphabets=[str(chr(ord('a')+i)) for i in range(26)]

# not really a static but we will just do this for now
SKIP_ZFS_TESTS = False

In [7]:
# parameters
startdisk_ = "c"
numberdisks_ = 2
zfsname_ = "tank"
raidmode_ = 0
# do we want the raid in +0 config, 
#   IE if we want raid50 with 2 raid 5 volumes in raid 0 config,
#   we have raidmode=5, raid0=2
raid0_ = 0

# zfs-options
ashift_ = "12"
compression_ = None # this may or may not work with our datasets, perhaps we can try lz4
recordsize_ = "32m"
atime_ = "off"

testname_ = "onedisk"
filesize_ = "100g"
runtime_ = "200"

In [8]:
def make_zfs(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime):
    raidmodestr = raidmode if type(raidmode)==str else RAID_CONFIGS[raidmode]
    cmdstr = "zpool create " + zfsname

    propertiesstr = " -o ashift="+ashift
    # we will worry about recordsize and atime later
    # each property has to have its own "-o"
    # this doesnt seem to work anyways, we need to set with "zfs set"
    #  + " recordsize="+recordsize + " atime="+atime
    # if compression:
    #     propertiesstr += " compression="+compression

    disks = []
    start = ord(startdisk[-1]) - ord('a')
    for i in range(start, start + numberdisks):
        j = i//26 - 1
        start = "" if j<0 else ALPHABET[j]
        disks.append ("sd" + start+ALPHABET[i%26])
    disksstr = ""
    for i, disk in enumerate(disks,0):
        if raid0!=0 and i%(numberdisks//raid0)==0 and i != 0:
            disksstr += " " + raidmodestr
        disksstr += " " + disk
    
    cmdstr += propertiesstr
    cmdstr += " " + raidmodestr
    cmdstr += disksstr
    return cmdstr

def make_fio_thruput(dir, testname, filesize, runtime, fs="zfs"):
    # writecmd = "fio --directory="+ dir                     +" --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize="+filesize+" --runtime="+runtime+" --time_based --group_reporting --name=throughput-test --eta-newline=1 >> " + testname + "/" + fs + "_write.txt"
    # readcmd  = "fio --filename="+ dir + "/throughput-test.0.0 --direct=1 --rw=read --bs=32m --ioengine=libaio --iodepth=64 --filesize="+filesize+" --runtime="+runtime+" --time_based --group_reporting --name=throughput-test --eta-newline=1 --readonly >> " + testname + "/" + fs + "_read.txt"
    writecmd = "fio --directory="+ dir                     +" --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize="+filesize+" --group_reporting --name=throughput-test --eta-newline=1 >> " + testname + "/" + fs + "_write.txt"
    readcmd  = "fio --filename="+ dir + "/throughput-test.0.0 --direct=1 --rw=read --bs=32m --ioengine=libaio --iodepth=64 --filesize="+filesize+" --group_reporting --name=throughput-test --eta-newline=1 --readonly >> " + testname + "/" + fs + "_read.txt"
    rmcmd = "rm "+ dir +"/throughput-test.0.0"
    return writecmd, readcmd, rmcmd

# need to make sure that:
# I don't know if this is needed? see 
# /etc/ldev.conf is changed:
# hostname - mgs     zfs:lustre-mgs/mgs
# hostname - mdt0    zfs:lustre-mdt0/mdt0
# hostname - ost0    zfs:lustre-ost0/ost0
# SELinux is disabled: sed -i '/^SELINUX=/s/.*/SELINUX=disabled/' /etc/selinux/config 
# see the vms for what we did with ldev.conf for remote connections
#
# lustre is started:
# systemctl start lustre
# ost dir should not 
def make_lustre(zfsname, mgsmdtname="mgsmdt", ip="$(hostname)", protocol="tcp"):
    #fs name is test btw
    full_ip = ip +"@"+ protocol
    cmds = []
    cmds.append("mkfs.lustre --backfstype=zfs --fsname=test --index=0 --reformat --mgs --mdt "+ mgsmdtname +"/mgsmdt") # mgsmdt should be static
    cmds.append("mkfs.lustre --backfstype=zfs --fsname=test --index=0 --ost --mgsnode="+ full_ip + " " + zfsname +"/ost0")
    # mount
    cmds.append("mount -t lustre " + mgsmdtname +"/mgsmdt /mnt/mgsmdt")
    cmds.append("mount -t lustre " + zfsname +"/ost0 /mnt/ost")
    cmds.append("mount -t lustre " + full_ip + ":/test" + " /mnt/lustre")

    # don't need this because its all mounted already. Should we remount? probably don't need to remount, but might as well... 
    # lustre client mount 
    # mount -t lustre <MGS node>
    #     /<fsname> <mount point>
    # ex:  mount -t lustre 192.168.169.207@tcp:/test /mnt/lustre
    return cmds

def clean(zfsname, mgsmdtname="mgsmdt", ip="$(hostname)", protocol="tcp"):
    full_ip = ip +"@"+ protocol
    devices = [mgsmdtname +"/mgsmdt", zfsname +"/ost0"]
    cmds = []
    # lustre
    cmds.append("umount -f "+ full_ip + ":/test")
    for device in devices:
        cmds.append("umount "+ device)
    for device in devices:
        cmds.append("tunefs.lustre --writeconf " + device)
    # zfs
    cmds.append ("zpool destroy -f "+ zfsname)
    return cmds
    
def make_echo(msg):
    line = "echo -------------------------------"
    return line, "echo "+msg, line

def print_title(msg):
    line = "-------------------------------"
    print (" "+line, "\n  ", msg, "\n", line)

def make_commands(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime, testname, ip, filesize, runtime):
    zfsdir = "/"+zfsname
    cmds = []
    cmds.append ("mkdir " + testname)
    
    # print_title ("make zfs")
    cmds.extend (make_echo("make zfs"))
    cmds.append (make_zfs(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime))
    
    if not SKIP_ZFS_TESTS:
        # print_title ("test zfs")
        cmds.extend (make_echo("test zfs"))
        cmds.extend (make_fio_thruput(zfsdir, testname, filesize, runtime, fs="zfs"))

    # print_title ("make lustre")
    cmds.extend (make_echo("make lustre"))
    lustrecmds = make_lustre(zfsname, ip=ip)
    cmds.extend (lustrecmds)

    cmds.extend (make_fio_thruput("/mnt/lustre", testname,filesize, runtime, fs="lustre"))

    cmds.extend (clean(zfsname, ip=ip))
    return cmds

In [9]:
./bench_fio --target /tank --type directory -o onedisk_fioplot_fine -b 32m -m write read --loops 1 --iodepth 1 --numjobs 1 --size 50gb --runtime 200 --quiet

SyntaxError: invalid syntax (Temp/ipykernel_22400/3324412632.py, line 1)

In [ ]:
def run_spec(startdisk, zfsname, csvfilename, ip, line):
    with open(csvfilename) as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        line += 2
        testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, runtime = [None]*10
        for _ in range(line):
            testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, runtime  = next(reader)
        numberdisks = int(numberdisks)
        raidmode = int(raidmode)
        raid0 = int(raid0)
        print (testname)
        return make_commands(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime, testname, ip, filesize, runtime)


In [ ]:
cmds = run_spec(startdisk_, zfsname_, "config.csv", "192.168.169.207", 0)
for cmd in cmds:
    print (cmd)

onedisk
mkdir onedisk
echo -------------------------------
echo make zfs
echo -------------------------------
zpool create tank -o ashift=12  sdc
echo -------------------------------
echo test zfs
echo -------------------------------
fio --directory=/tank --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1 >> onedisk/zfs_write.txt
fio --filename=/tank/throughput-test.0.0 --direct=1 --rw=read --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1 --readonly >> onedisk/zfs_read.txt
rm /tank/throughput-test.0.0
echo -------------------------------
echo make lustre
echo -------------------------------
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --reformat --mgs --mdt mgsmdt/mgsmdt
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --ost --mgsnode=192.168.169.207@tcp tank/ost0
mount -t lustre mgsmdt/mgsmdt /mnt/mgsmdt
mount -t lustre tank/os

In [ ]:
cmds = make_commands(startdisk_, numberdisks_, zfsname_, raidmode_, raid0_, ashift_, compression_, recordsize_, atime_, testname_, "192.168.169.207", filesize_, runtime_)
for cmd in cmds:
    print (cmd)

mkdir onedisk
echo -------------------------------
echo make zfs
echo -------------------------------
zpool create tank -o ashift=12  sdc sdd
echo -------------------------------
echo test zfs
echo -------------------------------
fio --directory=/tank --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1 >> onedisk/zfs_write.txt
fio --filename=/tank/throughput-test.0.0 --direct=1 --rw=read --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1 --readonly >> onedisk/zfs_read.txt
rm /tank/throughput-test.0.0
echo -------------------------------
echo make lustre
echo -------------------------------
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --reformat --mgs --mdt mgsmdt/mgsmdt
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --ost --mgsnode=192.168.169.207@tcp tank/ost0
mount -t lustre mgsmdt/mgsmdt /mnt/mgsmdt
mount -t lustre tank/ost0 /

# make sure the change device, name, rw
"fio --directory=" "--direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --runtime=120 --time_based --group_reporting --name=throughput-test --eta-newline=1"
"fio --directory=" "--direct=1 --rw=read --bs=32m --ioengine=libaio --iodepth=64 --filesize=100g --runtime=120 --time_based --group_reporting --name=throughput-test --eta-newline=1 --readonly"

In [ ]:
fio --directory=/tank --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=30g --numjobs=2 --group_reporting --name=throughput-test --eta-newline=1

./bench_fio --target /tank --type directory -o onedisk_fioplot -b 16m 32m 64m -m write --loops 1 --iodepth 1 2 --numjobs 1 2 --ss iops:10% --ss-ramp 10 --ss-dur 30 --runtime 300 --size 20g

./fio_plot/fio_plot -T "title" -r write -g -t bw -d 1 2 -n 1 2 -i benchmark_script/onedisk_fioplot/tank/32m/